In [60]:
# Manipulacion de datos
import numpy as np
import pandas as pd
import sqlite3

# Gráficos (???!)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 8)
import seaborn as sns
sns.set(style="darkgrid", font_scale=1.5)

# Ciencia de datos, machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import tree

# Modelos
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from xgboost.sklearn import XGBClassifier

# Leer/Guardar modelos
import joblib

/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [30]:
# Cargamos la BBDD en un dataframe pandas
con = sqlite3.connect("../data/ds_api.sqlite")
# Forzamos que todos los valores sean del tipo entero
data = pd.read_sql_query("SELECT * from desercion", con, coerce_float=True, dtype=np.int_)
#data.dtypes


In [31]:
# Separo en features y target
X = data.iloc[:,0:-1]
y = data.iloc[:,-1:]


In [32]:
# Escalamos los datos
scaler = StandardScaler()
scaler.fit(X)
data_escalada = scaler.transform(X)

X = pd.DataFrame(data_escalada, index=X.index, columns=X.columns)
#X.sample(15)

In [33]:
# Calculamos y guardamos standar deviation y mean
desviacion = data.std()
promedio = data.mean()

filename = '../data/desvio.dat'
joblib.dump(desviacion, filename)

filename = '../data/media.dat'
joblib.dump(promedio, filename)


['../data/media.dat']

In [34]:
# Separamos en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=50000)

In [35]:
# Probamos con regresion logistica
lg = LogisticRegression()
modelo_lg = lg.fit(X_train, y_train)
y_pred_test = modelo_lg.predict(X_test)

# matriz de confusión
conf_lg = confusion_matrix(y_test,y_pred_test)

/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
# guardamos el modelo y la matriz de confusion
filename = '../models/modelo_lg.mod'
joblib.dump(modelo_lg, filename)

filename = '../models/modelo_lg.mtx'
joblib.dump(conf_lg, filename)

['../models/modelo_lg.mtx']

In [39]:
# Probamos con naive bayes
nb = GaussianNB()
modelo_nb = nb.fit(X_train, y_train)
y_pred_test = modelo_nb.predict(X_test)

# matriz de confusión
conf_nb = confusion_matrix(y_test,y_pred_test)

/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
# guardamos el modelo y la matriz de confusion
filename = '../models/modelo_nb.mod'
joblib.dump(modelo_nb, filename)

filename = '../models/modelo_nb.mtx'
joblib.dump(conf_nb, filename)

['../models/modelo_nb.mtx']

In [61]:
# Probamos con Ada Boost
# n_estimators surgen despues de probar con varios numeros 
ab = AdaBoostClassifier(n_estimators=45, random_state=50000)
modelo_ab = ab.fit(X_train, y_train)
y_pred_test = modelo_ab.predict(X_test)

# matriz de confusión
conf_ab = confusion_matrix(y_test,y_pred_test)

/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
# guardamos el modelo y la matriz de confusion
filename = '../models/modelo_ab.mod'
joblib.dump(modelo_ab, filename)

filename = '../models/modelo_ab.mtx'
joblib.dump(conf_ab, filename)

['../models/modelo_ab.mtx']

In [67]:
# Probamos con XGBoost
# codigo y parametros descaradamente robado de Juan Domingo Gonzalez
xg = XGBClassifier(n_estimators=2000, learning_rate=0.02, n_jobs=4)
modelo_xg = xg.fit(X, y, early_stopping_rounds=5, eval_set=[(X_train, y_train)], verbose=False)
y_pred_test = modelo_xg.predict(X_test)

# matriz de confusión
conf_xg = confusion_matrix(y_test,y_pred_test)

/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/administrador/anaconda3/envs/ds_api/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to 

[14:44:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [69]:
# guardamos el modelo y la matriz de confusion
filename = '../models/modelo_xg.mod'
joblib.dump(modelo_xg, filename)

filename = '../models/modelo_xg.mtx'
joblib.dump(conf_xg, filename)

['../models/modelo_xg.mtx']

In [68]:
conf_xg

array([[23,  0],
       [ 0, 22]])